In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import heteropolymer_simulations as hs
import MDAnalysis as mda
import mdtraj as md
import numpy as np
import scipy as sp
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm

/jet/home/tfobe/.conda/envs/terphenyl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constructing a helix configuration for the MOP-terphenyl octamer system
In this notebook I will be using structural information from a MOP-terphenyl hexamer simulation to construct an octamer in an identified helix configuration. I'm moving this from a simple python script to a jupyter notebook to ease the development of this task.

Originally we constructed MOP-terphenyl helices by only using the a1, a2, p1, p2 and p3 torsion of the MOP-terphenyl system of an energy minimized structure. While we believed these to be the only flexible torsions in the system, setting these torsions from the helix identified in the hexamer system never quite matched the helical configuration very well. So in this notebook we will expand the construction to include all torsions present in a MOP-terphenyl monomer. Including other torsions such as the aromatic torsions should be able to capture slight bending of the aromatic groups that might be accurately captured from the original energy minimzed structure. 

In [48]:
# Get all torsions present in MOP hexamer simulation
hexamer_helix_cluster = mda.Universe("../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp", "../../../../hexamer_remd/t_250_450/clustering_output/cluster_23.xtc")

In [49]:
octamer_structure = mda.Universe("terphenyl_mop_octamer.itp", "em_octamer.gro")
octamer_structure

<Universe with 373 atoms>

In [50]:
# AtomGroup for residue 1
res1_oct = octamer_structure.select_atoms("resid 1")
res1_hex = hexamer_helix_cluster.select_atoms("resid 1")

In [51]:
# Appears that dihedral orders are preserved here despite atom names not being consistent

for i, (d_oct, d_hex) in enumerate(zip(res1_oct.dihedrals, res1_hex.dihedrals)):
    print("dihedral", i)
    print("octamer atoms: ", end="")
    for a in d_oct.atoms:
        print(a.name + " ", end="")
    print("\n", end="")
    print("hexamer atoms: ", end="")
    for a in d_hex.atoms:
        print(a.name + " ", end="")
    print("\n", end="")

dihedral 0
octamer atoms: C1 C2 C3 C4 
hexamer atoms: C1 C2 C3 C4 
dihedral 1
octamer atoms: C1 C2 C3 H1 
hexamer atoms: C1 C2 C3 H1 
dihedral 2
octamer atoms: C1 C2 C22 C21 
hexamer atoms: C1 C2 C22 C21 
dihedral 3
octamer atoms: C1 C2 C22 H19 
hexamer atoms: C1 C2 C22 H19 
dihedral 4
octamer atoms: C1 O17 C177 H153 
hexamer atoms: C1 O13 C133 H115 
dihedral 5
octamer atoms: C1 O17 C177 H154 
hexamer atoms: C1 O13 C133 H116 
dihedral 6
octamer atoms: C1 O17 C177 H155 
hexamer atoms: C1 O13 C133 H117 
dihedral 7
octamer atoms: O1 C1 C2 C3 
hexamer atoms: O1 C1 C2 C3 
dihedral 8
octamer atoms: O1 C1 C2 C22 
hexamer atoms: O1 C1 C2 C22 
dihedral 9
octamer atoms: O1 C1 O17 C177 
hexamer atoms: O1 C1 O13 C133 
dihedral 10
octamer atoms: C2 C1 O17 C177 
hexamer atoms: C2 C1 O13 C133 
dihedral 11
octamer atoms: C2 C3 C4 C5 
hexamer atoms: C2 C3 C4 C5 
dihedral 12
octamer atoms: C2 C3 C4 H2 
hexamer atoms: C2 C3 C4 H2 
dihedral 13
octamer atoms: C2 C22 C21 C5 
hexamer atoms: C2 C22 C21 C5 
di

In [7]:
# Now we can extract the list of all torsions defined in the octamer residue
octamer_r1_dihes = []
for i, d_oct in enumerate(res1_oct.dihedrals):
    dihe_oct = []
    if any([atom.resname != "OCT" for atom in d_oct.atoms]):
        continue
    if any([atom.resid != 1 for atom in d_oct.atoms]):
        continue
    for atom in d_oct.atoms:
        dihe_oct.append(atom.name)
    octamer_r1_dihes.append(dihe_oct)

In [8]:
hexamer_r1_dihes = []
for i, d_hex in enumerate(res1_hex.dihedrals):
    dihe_hex = []
    if any([atom.resname != "HEX" for atom in d_hex.atoms]):
        continue
    if any([atom.resid != 1 for atom in d_hex.atoms]):
        continue
    for atom in d_hex.atoms:
        dihe_hex.append(atom.name)
    hexamer_r1_dihes.append(dihe_hex)

In [9]:
# Using a function in heteropolymer_simulations we can extract the torsions from the entire residue
all_octamer_torsions = []
for torsion_id in octamer_r1_dihes:
    t_ids = hs.utils.get_torsion_ids(octamer_structure, "OCT", torsion_id)
    all_octamer_torsions.append(t_ids)

In [10]:
all_hexamer_torsions = []
for torsion_id in hexamer_r1_dihes:
    t_ids = hs.utils.get_torsion_ids(hexamer_helix_cluster, "HEX", torsion_id)
    all_hexamer_torsions.append(t_ids)

In [11]:
# List of all torsion ids within dihedral object
all_torsion_ids = [[d.atoms[0].name, d.atoms[1].name, d.atoms[2].name, d.atoms[3].name,] for d in hexamer_helix_cluster.dihedrals]
len(all_torsion_ids)

721

In [47]:
# Now we extract all torsions of the helical segment in the hexamer

plot_distributions = False

ideal_torsions = []
avg_torsions = []
for i in tqdm(range(len(all_hexamer_torsions))):
    torsion_values = []
    # Pull torsions from residues 4,5,6 for all frames
    for ts in hexamer_helix_cluster.trajectory:
        hexamer_helix_cluster.atoms.positions = ts.positions
        for torsion_id in all_hexamer_torsions[i][-3:]:
            torsion_index = all_torsion_ids.index(torsion_id)
            torsion_values.append(hexamer_helix_cluster.dihedrals[torsion_index].value())
    # Plot histogram
    if "H" not in " ".join(all_hexamer_torsions[i][0]) and plot_distributions:
        plt.figure()
        bin_edges = np.linspace(-180, 180, 180)
        plt.hist(torsion_values, bins = bin_edges, density = True)
        plt.title("Torsion " + " ".join(all_hexamer_torsions[i][0]))
        plt.xlabel("Torsion " + str(i) + "(radians)")
        plt.ylabel("Density")
    
    # Save histogram max value
    hist, bin_edges_out = np.histogram(torsion_values, bins = bin_edges, density = True)
    bin_centers = [(bin_edges[i] + bin_edges[i+1])/2 for i in range(len(bin_edges) - 1)]
    torsion_max_density = bin_centers[np.argmax(hist)]
    ideal_torsions.append(torsion_max_density)
    
ideal_torsions = np.array(ideal_torsions)

100%|█████████████████████████████████████████| 105/105 [01:58<00:00,  1.13s/it]


In [55]:
# Compare BAT torsions to Universe torsions
ice = hs.edit_conf.InternalCoordinateEditor("../../../../hexamer_remd/t_250_450/build_system/em_mop_hexamer.gro", "../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp")
test_u = mda.Universe("../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp", "../../../../hexamer_remd/t_250_450/build_system/em_mop_hexamer.gro")

/jet/home/tfobe/.conda/envs/terphenyl/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:903: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


<TopologyGroup containing 721 dihedrals>

In [108]:
test_all_torsions = [[atom.name for atom in dihe.atoms] for dihe in test_u.dihedrals]
test_all_torsions

[['C1', 'C2', 'C3', 'C4'],
 ['C1', 'C2', 'C3', 'H1'],
 ['C1', 'C2', 'C22', 'C21'],
 ['C1', 'C2', 'C22', 'H19'],
 ['C1', 'O13', 'C133', 'H115'],
 ['C1', 'O13', 'C133', 'H116'],
 ['C1', 'O13', 'C133', 'H117'],
 ['O1', 'C1', 'C2', 'C3'],
 ['O1', 'C1', 'C2', 'C22'],
 ['O1', 'C1', 'O13', 'C133'],
 ['C2', 'C1', 'O13', 'C133'],
 ['C2', 'C3', 'C4', 'C5'],
 ['C2', 'C3', 'C4', 'H2'],
 ['C2', 'C22', 'C21', 'C5'],
 ['C2', 'C22', 'C21', 'H18'],
 ['C3', 'C2', 'C1', 'O13'],
 ['C3', 'C2', 'C22', 'C21'],
 ['C3', 'C2', 'C22', 'H19'],
 ['C3', 'C4', 'C5', 'C6'],
 ['C3', 'C4', 'C5', 'C21'],
 ['C4', 'C3', 'C2', 'C22'],
 ['C4', 'C5', 'C6', 'C7'],
 ['C4', 'C5', 'C6', 'C12'],
 ['C4', 'C5', 'C21', 'C22'],
 ['C4', 'C5', 'C21', 'H18'],
 ['C5', 'C4', 'C3', 'H1'],
 ['C5', 'C6', 'C7', 'C8'],
 ['C5', 'C6', 'C7', 'C13'],
 ['C5', 'C6', 'C12', 'C11'],
 ['C5', 'C6', 'C12', 'H8'],
 ['C5', 'C21', 'C22', 'H19'],
 ['C6', 'C5', 'C4', 'H2'],
 ['C6', 'C5', 'C21', 'C22'],
 ['C6', 'C5', 'C21', 'H18'],
 ['C6', 'C7', 'C8', 'C9'],
 

In [111]:
for i in range(len(ice.torsion_ids)):
    print("index:", i)
    print("ICE torsion:", ice.torsion_ids[i], ice.torsions[i] * 180 / np.pi)
    if ice.torsion_ids[i].split(" ") in test_all_torsions:
        t_index = test_all_torsions.index(ice.torsion_ids[i].split(" "))
        print("MDA Universe:", " ".join(test_all_torsions[t_index]), test_u.dihedrals[t_index].value())
    if ice.torsion_ids[i].split(" ")[::-1] in test_all_torsions:
        t_index = test_all_torsions.index(ice.torsion_ids[i].split(" ")[::-1])
        print("MDA Universe:", " ".join(test_all_torsions[t_index]), test_u.dihedrals[t_index].value())
    else:
        print("Torsion not present in MDA Universe")

index: 0
ICE torsion: C135 O14 C138 O15 -4.108886978165111
MDA Universe: C135 O14 C138 O15 -4.108886978165118
Torsion not present in MDA Universe
index: 1
ICE torsion: C134 C135 O14 C138 -178.12294664824458
MDA Universe: C134 C135 O14 C138 -178.12294664824458
Torsion not present in MDA Universe
index: 2
ICE torsion: H118 C134 C135 O14 -179.81399685338377
MDA Universe: O14 C135 C134 H118 -179.81399685338377
index: 3
ICE torsion: H119 C134 C135 O14 120.14894112284682
MDA Universe: O14 C135 C134 H119 -59.66505573053691
index: 4
ICE torsion: H120 C134 C135 O14 -120.25601147070904
MDA Universe: O14 C135 C134 H120 59.92999167590721
index: 5
ICE torsion: N6 C138 O14 C135 179.24462418305654
MDA Universe: N6 C138 O14 C135 175.13573720489143
Torsion not present in MDA Universe
index: 6
ICE torsion: C136 C135 C134 H118 120.26403906479784
MDA Universe: C136 C135 C134 H118 -59.549957788585935
Torsion not present in MDA Universe
index: 7
ICE torsion: C137 C135 C134 H118 -120.08098418091248
MDA Unive

MDA Universe: C50 C51 C57 C58 50.11352322214117
Torsion not present in MDA Universe
index: 93
ICE torsion: C49 C50 C51 C57 5.220568830282693
MDA Universe: C49 C50 C51 C57 5.220568830282685
Torsion not present in MDA Universe
index: 94
ICE torsion: C48 C49 C50 C51 47.241659701173845
MDA Universe: C48 C49 C50 C51 47.241659701173845
Torsion not present in MDA Universe
index: 95
ICE torsion: H40 C48 C49 C50 -1.1856177739770148
MDA Universe: C50 C49 C48 H40 -1.1856177739770244
index: 96
ICE torsion: C47 C48 C49 C50 178.7731131878142
MDA Universe: C47 C48 C49 C50 177.5874954138372
Torsion not present in MDA Universe
index: 97
ICE torsion: H39 C47 C48 H40 -0.6275868038417959
MDA Universe: H39 C47 C48 H40 -0.6275868038417873
Torsion not present in MDA Universe
index: 98
ICE torsion: C46 C47 C48 H40 179.68602186730476
MDA Universe: C46 C47 C48 H40 179.05843506346292
Torsion not present in MDA Universe
index: 99
ICE torsion: C45 C46 C47 H39 0.8994069578971313
MDA Universe: C45 C46 C47 H39 0.8994

MDA Universe: H108 C129 C130 H110 -171.96023416539666
index: 187
ICE torsion: H111 C130 C129 H108 -121.52345346865953
MDA Universe: H108 C129 C130 H111 66.51631236594382
index: 188
ICE torsion: H112 C130 C129 H108 119.73864948135217
MDA Universe: H108 C129 C130 H112 -52.221584684044494
index: 189
ICE torsion: H107 C128 C123 C117 2.857524212213663
MDA Universe: C117 C123 C128 H107 2.857524212213667
index: 190
ICE torsion: H98 C118 C117 C116 -178.98572289454606
MDA Universe: C116 C117 C118 H98 -178.98572289454606
index: 191
ICE torsion: C119 C118 C117 C116 178.16726009469292
MDA Universe: C116 C117 C118 C119 -0.8184627998531471
index: 192
ICE torsion: H103 C122 C116 C115 -1.9967506679417766
MDA Universe: C115 C116 C122 H103 -1.9967506679417837
index: 193
ICE torsion: C121 C122 C116 C115 179.54993922609862
MDA Universe: C115 C116 C122 C121 177.55318855815688
index: 194
ICE torsion: H113 C131 C115 C114 -179.44641220542803
MDA Universe: C114 C115 C131 H113 -179.44641220542803
index: 195
ICE

MDA Universe: C9 O2 C10 H5 64.01909332988593
index: 281
ICE torsion: H6 C10 O2 C9 119.62172187757788
MDA Universe: C9 O2 C10 H6 -56.98268569955781


In [13]:
# InternalCoordinateEditor for Hexamer
# We run these together to ensure we start with a minimized structure

ice = hs.edit_conf.InternalCoordinateEditor("../../../../hexamer_remd/t_250_450/build_system/em_mop_hexamer.gro", "../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp")

for i, res_torsions in enumerate(all_hexamer_torsions):
    for res_torsion in res_torsions:
        torsion_ids, torsion_values = ice.find_torsions(res_torsion)

        if len(torsion_ids) != 0:
            print(torsion_ids[0])
            
            if " ".join(res_torsion) == torsion_ids[0]:
                ice.set_torsion(torsion_ids[0], (ideal_torsions[i]) * np.pi / 180)
            else:
                ice.set_torsion(torsion_ids[0], (180 - ideal_torsions[i]) * np.pi / 180)
            ice.update_internal_coordinates()
ice.write_structure("hexamer_helix.gro")

/jet/home/tfobe/.conda/envs/terphenyl/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:903: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


C1 C2 C3 H1
C23 C24 C25 H20
C45 C46 C47 H39
C67 C68 C69 H58
C89 C90 C91 H77
C111 C112 C113 H96
H19 C22 C2 C1
H38 C44 C24 C23
H57 C66 C46 C45
H76 C88 C68 C67
H95 C110 C90 C89
H114 C132 C112 C111
O1 C1 C2 C3
O3 C23 C24 C25
O5 C45 C46 C47
O7 C67 C68 C69
O9 C89 C90 C91
O11 C111 C112 C113
C22 C2 C1 O1
C2 C3 C4 H2
C24 C25 C26 H21
C46 C47 C48 H40
C68 C69 C70 H59
C90 C91 C92 H78
C112 C113 C114 H97
C3 C4 C5 C6
C25 C26 C27 C28
C47 C48 C49 C50
C69 C70 C71 C72
C91 C92 C93 C94
C113 C114 C115 C116
C4 C5 C6 C7
C26 C27 C28 C29
C48 C49 C50 C51
C70 C71 C72 C73
C92 C93 C94 C95
C114 C115 C116 C117
C12 C6 C5 C4
C34 C28 C27 C26
C56 C50 C49 C48
C78 C72 C71 C70
C100 C94 C93 C92
C122 C116 C115 C114
H18 C21 C5 C4
H37 C43 C27 C26
H56 C65 C49 C48
H75 C87 C71 C70
H94 C109 C93 C92
H113 C131 C115 C114
C8 C7 C6 C5
C30 C29 C28 C27
C52 C51 C50 C49
C74 C73 C72 C71
C96 C95 C94 C93
C118 C117 C116 C115
C5 C6 C7 C13
C27 C28 C29 C35
C49 C50 C51 C57
C71 C72 C73 C79
C93 C94 C95 C101
C115 C116 C117 C123
C11 C12 C6 C5
C33 C34 C2

In [14]:
for t_id, value in zip(hexamer_r1_dihes, ideal_torsions):
    if "C1" in t_id and "C2" in t_id:
        print(" ".join(t_id), round(value, 2))

C1 C2 C3 C4 -178.99
C1 C2 C3 H1 0.0
C1 C2 C22 C21 178.99
C1 C2 C22 H19 2.01
O1 C1 C2 C3 -168.94
O1 C1 C2 C22 8.04


In [23]:
for t_id in ice.torsion_ids:
    if "O1" in t_id.split(" "):
        print(t_id, ice.get_torsion(t_id) * 180 /np.pi)

O1 C1 C2 C3 -179.95477389980888
C22 C2 C1 O1 -179.94404914393294


In [45]:
hexamer_helix_cluster.positions.positions = ice.universe.coord.positions

In [46]:
mda.lib.distances.calc_dihedrals(
    hexamer_helix_cluster.select_atoms("name C22").positions,
    hexamer_helix_cluster.select_atoms("name C2").positions,
    hexamer_helix_cluster.select_atoms("name C1").positions,
    hexamer_helix_cluster.select_atoms("name O1").positions,
) * 180 / np.pi

array([-22.00695762])

In [44]:
print(ice.find_torsions(["O1", "C2", "C1", "C22"]))
ice.set_torsion("C22 C2 C1 O1", np.pi)
ice.update_internal_coordinates()
ice.write_structure("test.gro")
print(ice.find_torsions(["O1", "C2", "C1", "C22"]))

(['C22 C2 C1 O1'], [-3.1406161269321133])
(['C22 C2 C1 O1'], [3.141592653589793])


In [18]:
ice2 = hs.edit_conf.InternalCoordinateEditor("test.gro", "../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp")
ice2.find_torsions(["O1", "C2", "C1", "C22"])

(['C22 C2 C1 O1'], [-3.1304828710314023])

In [19]:
ice.ic_list

array([ 5.11999969e+01,  4.74900017e+01,  5.96299973e+01,  1.72654879e+00,
        1.64414167e+00,  1.52384609e+00,  1.22812355e+00,  1.35517716e+00,
        2.20643711e+00,  1.43630930e+00,  1.52807192e+00,  1.09959091e+00,
        1.09827241e+00,  1.09228798e+00,  1.36656919e+00,  1.54654440e+00,
        1.53476356e+00,  1.01198039e+00,  1.45859346e+00,  1.09498359e+00,
        1.09077636e+00,  1.09242508e+00,  1.09334564e+00,  1.09572067e+00,
        1.09594149e+00,  1.10077366e+00,  1.52085421e+00,  1.39380491e+00,
        1.09046416e+00,  1.39362434e+00,  1.08613864e+00,  1.39521080e+00,
        1.08231055e+00,  1.40267791e+00,  1.48343950e+00,  1.41753394e+00,
        1.47081649e+00,  1.41141615e+00,  1.07800378e+00,  1.39746696e+00,
        1.09288190e+00,  1.40836992e+00,  1.48438439e+00,  1.38296571e+00,
        1.22743452e+00,  1.01809205e+00,  1.45540479e+00,  1.09936322e+00,
        1.51874998e+00,  1.40474230e+00,  1.08604730e+00,  1.39785133e+00,
        1.08926758e+00,  

In [20]:
ice = hs.edit_conf.InternalCoordinateEditor("../../../../hexamer_remd/t_250_450/build_system/em_mop_hexamer.gro", "../../../../hexamer_remd/t_250_450/terphenyl_mop_hexamer.itp")
